In [1]:
import json
import re
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import concurrent.futures

# Generate BFC-BIC link dataset

This notebook perform the generation of the dataset of pairs (links) between the commit that fixed a bug (BFC) and the commit that introduced that bug.

To do this, all the commits in the commit dataset (~1.2M) will be loaded lazily, and requires ~8GB of RAM memory to be able to mine BFCs<->BICs. Although in some moments of the execution it is possible to reach a usage of 20GB-30GB due to the multiple Threads that load information in memory and then release it.

This notebook was executed on an Ubuntu 22.04 machine with 64 CPU cores and 116 GB of RAM.

In [2]:
def getCommits():
    with open('../linux-commits-2023-11-12.json') as f:
        for commit in f:
            yield json.loads(commit)

To efficiently calculate the distance between a BFC and a BIC, the project repository is required, which can be obtained by uncommenting the following line and executing it.

In [3]:
# ! git clone git://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git

In [4]:
all_commits_map = {}
all_commits_list = []
for commit in getCommits():
    
    commit_reduced = {
        "hash": commit['data']['commit'],
        "message": commit['data']['message'] if "message" in commit['data'] else None,
        "date": commit['data']['CommitDate']
    }

    all_commits_list.append(commit_reduced)
    
    for n in [5, 6, 7, 8, 9, 10, 11, 12]:
        # To cover collisions
        if commit_reduced['hash'][0:n] in all_commits_map:
            all_commits_map[commit_reduced['hash'][0:n]].append(commit_reduced)
        else:
            all_commits_map[commit_reduced['hash'][0:n]] = [commit_reduced]

In [5]:
date_format = '%a %b %d %H:%M:%S %Y %z'
def datesDistance(bfc_date, bic_date):
    bfc_date_formated = datetime.strptime(bfc_date, date_format)
    bic_date_formated = datetime.strptime(bic_date, date_format)
    return bfc_date_formated - bic_date_formated

**Note**: Go to the end of the document for an alternative to the following method

In [6]:
def commitDistance(bfc_hash, bic_hash):
    distance_raw = subprocess.getoutput("cd linux/ && git rev-list "+bfc_hash+"..."+bic_hash+" | wc -l")
    return int(distance_raw)

In [7]:
def analyzeCornerCases(commit):
    global http_links

    commit_message = commit['message'].replace('"','').replace('(',' ').replace(')',' ')
    match = re.search(r"Fixes:[^\S\n]+(\w+)", commit_message)
    if match is not None:
        bic_hash = match.group(1)
    else:
        return None
        
    # There are fixes with a link to Bugzilla/Other sites
    if bic_hash.startswith("http"):
        http_links+=1
        return None

    if bic_hash in [
        'IRQ','NB','SLI','Bug','line','tag',
        'tags','Discovery','discovery','drivers','igt','Bugzilla','bugzilla',
        'correctly','computation','terminate','Configure','addresses',
        'hashes', 'second'
    ]:
        return None

    if bic_hash == "commit":
        match = re.search(r"Fixes:[^\S\n]+commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        bic_hash = match.group(1)

    elif bic_hash == "Commit":
        match = re.search(r"Fixes:[^\S\n]+Commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        bic_hash = match.group(1)

    # Double 'Fixes: ' (7 cases)
    elif bic_hash == "Fixes":
        match = re.search("Fixes: Fixes:[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        if match is not None:
            bic_hash = match.group(1) 

    # Special format (3 cases)
    elif bic_hash == "linux":
        match = re.search("Fixes: linux-next commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        if match is not None:
            bic_hash = match.group(1)

    # 77 cases, no commits hashes (manually checked)
    elif len(bic_hash[0:12]) < 6:
        return None

    return bic_hash

In [8]:
def analyzeCommit(commit):
    global errors
    global not_in_git
    global not_sha
    global perfect_samples
    global short_commit_hash
    global corner_cases
    
    try:
        # There are commits without message
        if commit['message'] is None:
            return None

        bic_hash = ""

        # Search: "Fixes: <hash>"
        match = re.search(r"Fixes:[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit['message'])
        if match is not None:
            bic_hash = match.group(1)
            perfect_samples+=1
        else:
            #return None
            # Search: "Fixes <something>"
            bic_hash = analyzeCornerCases(commit)
            if bic_hash is None: return None
            corner_cases+=1

        # Last check, if commit hash not in map, discard it
        if bic_hash[0:12] not in all_commits_map:
            not_in_git+=1
            return None
                
        candidates = all_commits_map[bic_hash[0:12]]

        if len(candidates) > 1:
            print("Collision", bic_hash[0:12], candidates[0]['hash'],candidates[1]['hash'], "BFC: ", commit['hash'])
            return None
        else:
            bic = candidates[0]
        
        delta = datesDistance(commit['date'],bic['date'])
        c_distance = commitDistance(commit['hash'], bic['hash'])
        result = {
            'BFC_hash': commit['hash'],
            'BIC_hash':  bic['hash'],
            'BFC_comment': commit['message'].split('\n', 1)[0],
            'BIC_comment': bic['message'].split('\n', 1)[0],
            'daysDistance': delta.days,
            'commitDistance': c_distance
        }
        return result

    except Exception as e:
        errors+=1
        match = re.search("Fixes: (.*)", commit['message'])    
        print("Error matching: ",match.group(0), "BFC:",commit['hash'])
        return None

In [9]:
links = []
errors = 0
corner_cases = 0
http_links = 0
not_in_git = 0
perfect_samples = 0

future_results = []
with concurrent.futures.ThreadPoolExecutor(64) as executor:
    for commit in all_commits_list:
        future = executor.submit(analyzeCommit, commit)
        future_results.append(future)
        
for future in future_results:
    try:
        result = future.result() 
        if result is not None:
            links.append(result)
    except Exception as e:
        print(e)

print("http_links",http_links)
print("not_in_git",not_in_git)
print("perfect_samples",perfect_samples)
print("corner_cases",corner_cases)
print("errors",errors)

Collision 22a5dc 22a5dc0e5e3e8fef804230cd73ed7b0afd4c7bae 22a5dc10e3f8fb8370748ea19dc4e3e1620d8296 BFC:  d7924450e14ea414568563ec01489f77452b00b4
Collision 4f1982 4f198289747f0391bc5a5574279b1791a8ca2d06 4f1982b4e262c45475a91b4253e9bc7f7c991c13 BFC:  3fd61b209977db8a9fe6c44d5a5a7aee7a255f64
Error matching:  Fixes: commit BFC: 7ea38c6c3622bc65279dc6a1fecd28227027fbb5
Error matching:  Fixes: commit 8700e3e7c48A5 ("Add Soft RoCE driver") BFC: e259934d4df7f99f2a5c2c4f074f6a55bd4b1722
http_links 139
not_in_git 1163
perfect_samples 91959
corner_cases 422
errors 2


In [10]:
len(links)

91216

In [11]:
filtered_links = []
for link in links:
    if link['BIC_hash'] != '1da177e4c3f41524e886b7f1b8a0c1fc7321cac2':
        filtered_links.append(link)
len(filtered_links)

90760

In [12]:
df = pd.DataFrame(filtered_links)
df.to_csv('bfc_bic.csv', index=False)  

**Note**: Its possible to calculate  commitDistance using only the dataset, but for performance reasons, its faster doing using the command `git rev-list` on the Linux Kernel Repository. An alternative code is provided:

```python
def find_path(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start in graph.keys():
        return None
    for parent in getParents(start):
        if parent not in path:
            newpath = find_path(graph, parent, end, path)
            if newpath is not None: return newpath
        return None
def getParents(_hash):
    return all_commits_map[_hash[0:12]][0]['data']['parents']
def commitDistance(bfc_hash, bic_hash):
    return len(find_path(all_commits_map, bfc_hash, bic_hash))-1
```